### Homework #1
Artem Chernitsa, B20-AI, a.chernitsa@innopolis.university

# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [1]:
import argparse
import os
import re
import requests


# def wget(url, filename):
#     # allow redirects - in case file is relocated
#     resp = requests.get(url, allow_redirects=True)
#     # this can also be 2xx, but for simplicity now we stick to 200
#     # you can also check for `resp.ok`
#     if resp.status_code != 200:
#         print(resp.status_code, resp.reason, 'for', url)
#         return
    
#     # just to be cool and print something
#     print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
#     print()
    
#     # try to extract filename from url
#     if filename is None:
#         # start with http*, ends if ? or # appears (or none of)
#         m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
#         filename = m.group(1)
#         if not filename:
#             raise NameError(f"Filename neither given, nor found for {url}")

#     # what will you do in case 2 websites store file with the same name?
#     if os.path.exists(filename):
#         raise OSError(f"File {filename} already exists")
    
#     with open(filename, 'wb') as f:
#         f.write(resp.content)
#         print(f"File saved as {filename}")


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description='download file.')
#     parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
#     parser.add_argument("url", type=str, default=None, help="Provide URL here")
#     args = parser.parse_args()
#     wget(args.url, args.filename)

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [2]:
import requests
from urllib.parse import quote
import os

class Document:
    
    def __init__(self, url):
        self.url = url
        self.file_name = quote(url, safe='')
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        try:
            response = requests.get(self.url, timeout=(5, 30))
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False
    
    def persist(self):
        with open(self.file_name, "wb") as f:
            f.write(self.content)
            
    def load(self):
        if os.path.exists(self.file_name):
            with open(self.file_name, "rb") as f:
                self.content = f.read()
            return True
        else:
            return False


In [3]:
# requests.head(some_link)
# header = h.headers
# content_type = header.get('content-type')

### 1.1.1. Tests ###

In [4]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

In [5]:
doc = Document('https://cdn.drivemusic.me/dl/online/ZhzI3EFVh2ScpkkYhtRf4Q/1675321813/download_music/2014/05/nico-vinz-am-i-wrong.mp3')
doc.get()

## 1.2. [10] Parse HTML
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
1. `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
2. `self.images` list of images met in a document. Again, links can be relative to current page.
3. `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [6]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):
    
    def parse(self):
        #TODO extract plain text, images and links from the document
        self.anchors = [("fake link text", "http://fake.url/")]
        self.images = ["http://image.com/fake.jpg"]
        self.text = "fake text and some other text"

In [7]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):
    
    def parse(self):
        soup = BeautifulSoup(self.content, 'html.parser', from_encoding='utf-8')

        # extract links
        self.anchors = []
        for link in soup.find_all('a'):
            text = link.text
            href = link.get('href')
            # fix relative links using urllib.parse.urljoin()
            url = urllib.parse.urljoin(self.url, href)
            self.anchors.append((text, url))

        # extract images
        self.images = []
        for img in soup.find_all('img'):
            src = img.get('src')
            # fix relative links using urllib.parse.urljoin()
            url = urllib.parse.urljoin(self.url, src)
            self.images.append(url)

        # extract plain text
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
        texts = soup.findAll(text=True)
        visible_texts = filter(tag_visible, texts)
        self.text = " ".join(t.strip() for t in visible_texts)

### 1.2.1. Tests

In [8]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

In [9]:
print(*doc.anchors, sep='\n')

('telegram', 'https://t.me/sprotasov')
('email', 'mailto:stanislav.protasov@gmail.com')
('Curriculum vitae', 'https://docs.google.com/document/d/e/2PACX-1vQqlsxmlbkwp7CypdNg5vcl9zEfE1w6EFppJ2iBbHpZrpOI0AIzFkeu21-Or1_PYlnq1ICyLR1qaNlu/pub')
('Google Scholar', 'https://scholar.google.ru/citations?user=pDske8oAAAAJ')
('GitHub', 'https://github.com/str-anger')
('Track record in Quantum', 'http://sprotasov.ru/q.html')
('ResearchGate', 'https://www.researchgate.net/profile/Stanislav-Protasov')
('Публикации в eLibrary', 'http://elibrary.ru/author_items.asp?authorid=789317')
('Facebook', 'https://www.facebook.com/stanislav.protasov')
('LinkedIn', 'https://www.linkedin.com/pub/stanislav-protasov/28/651/b38')
('Research with Stas telegram channel', 'https://t.me/iu_aml')
('Подкаст "Происхождение видов": telegram', 'https://t.me/origin_of_species')
('iTunes', 'https://itunes.apple.com/ru/podcast/происхождение-видов/id1282666034')
('RSS', 'http://sprotasov.ru/podcast/rss.xml')
('Automatic testing 

## 1.3. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria to succeed in the task**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained from inside `<body>` tag only.

In [10]:
from collections import Counter

class HtmlDocumentTextData:
    def __init__(self, url):
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        # Get text within <body> tag only
        soup = BeautifulSoup(self.doc.content, 'html.parser', from_encoding='utf-8')
        body = soup.find("body")
        text = body.get_text(" ", strip=True) if body else ""

        # split text into sentences using regex
        pattern = re.compile(r"[.!?][\s]+")
        sentences = pattern.split(text)

        return sentences

    def get_word_stats(self):
        words = []
        for sentence in self.get_sentences():
            # Flatten list
            words += re.findall(r'\b\w+\b', sentence.lower())
            # words.extend(sentence.lower().split())

        # words = re.findall(r'\b\w+\b', self.text.lower())

        return Counter(words)

### 1.3.1. Tests ###

In [11]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

[('и', 46), ('в', 23), ('иннополис', 22), ('на', 14), ('с', 14), ('университет', 12), ('университета', 12), ('по', 10), ('ит', 10), ('центр', 10)]


## 1.4. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [12]:
import urllib.parse

def parse_robots_txt(base_url):
    robots_url = urllib.parse.urljoin(base_url, '/robots.txt')
    response = requests.get(robots_url)
    content = response.text

    disallowed_urls = []
    for line in content.splitlines():
        if line.startswith('Disallow:'):
            path = line.split(':', 1)[1].strip()
            if path:
                disallowed_urls.append(urllib.parse.urljoin(base_url, path))
    return set(disallowed_urls)

def get_base_url(full_url):
    parsed_url = urllib.parse.urlparse(full_url)
    base_url = parsed_url.scheme + "://" + parsed_url.netloc
    return base_url

In [13]:
from queue import Queue
from hashlib import md5

class Crawler:
    
    def crawl_generator(self, source, depth=1):
        visited = set()
        queue = Queue()
        queue.put((source, 0))

        base_url = get_base_url(source)
        visited = visited.union(parse_robots_txt(base_url))

        while not queue.empty():
            url, d = queue.get()
            url_hash = md5(url.encode()).hexdigest()
            if url_hash in visited:
                continue
            visited.add(url_hash)
            
            # print(f'I process this url: {url}')

            try:
                doc = HtmlDocumentTextData(url)
            except:
                continue

            if d == depth:
                continue
            
            yield doc

            # print(f'd = {d}')

            # print(len(doc.doc.anchors))
            for _, link in doc.doc.anchors:
                queue.put((link, d + 1))


### 1.4.1. Tests ###

In [14]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

https://innopolis.university/en/
339 distinct word(s) so far
https://innopolis.university/
876 distinct word(s) so far
https://apply.innopolis.university/en
1496 distinct word(s) so far
https://innopolis.university/lk/
1505 distinct word(s) so far
https://innopolis.university/en/about/
1685 distinct word(s) so far
https://innopolis.university/en/board/
1762 distinct word(s) so far
https://innopolis.university/en/team/
1763 distinct word(s) so far
https://innopolis.university/en/team-structure/
1766 distinct word(s) so far
https://innopolis.university/en/team-structure/education-academics/
1768 distinct word(s) so far
https://innopolis.university/en/team-structure/techcenters/
1773 distinct word(s) so far
https://innopolis.university/en/faculty/
2642 distinct word(s) so far
https://career.innopolis.university/en/job/
3088 distinct word(s) so far
https://career.innopolis.university/en/
3197 distinct word(s) so far
https://innopolis.university/en/campus
3310 distinct word(s) so far
https:

https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
Skipping https://innopolis.university/public/files/Consent_to_the_processing_of_PD_for_UI.pdf
https://t.me/universityinnopolis
12137 distinct word(s) so far
https://vk.com/innopolisu
12319 distinct word(s) so far
https://www.youtube.com/user/InnopolisU
13970 distinct word(s) so far
https://apply.innopolis.ru/en/
15568 distinct word(s) so far
https://panoroo.com/virtual-tours/NvQZM6B2
15590 distinct word(s) so far
https://media.innopolis.university/en/events/
15592 distinct word(s) so far
https://minobrnauki.gov.ru/
15798 distinct word(s) so far
https://career.innopolis.university/konkursnyezayavkiprofessorskoprepodavatelskogosostava/
15821 distinct word(s) so far


Done
[('1', 2821), ('the', 2730), ('and', 2525), ('of', 2268), ('0', 1854), ('2', 1759), ('to', 1454), ('in', 1418), ('university', 1244), ('3', 1237), ('4', 1046), ('a', 1043), ('innopolis', 908), ('50', 848), ('for', 821), ('i', 767), ('5', 707), ('fill', 606), ('6', 600), ('7', 568)]


In [15]:
# url = r'https://career.innopolis.university/konkursnyezayavkiprofessorskoprepodavatelskogosostava/'
# url = 'https://career.innopolis.university/public/files/Согласие на обработку ПДн для УИ.pdf'
# doc = HtmlDocument(url)
# doc.get()
# doc.parse()
# doc.anchors